<h1>Importamos paquetes</h1>

In [ ]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *
from time import *
from statistics import mode
import pandas as pd
import matplotlib.pyplot as plt
import requests
import socket
from tqdm import tqdm
import numpy as np
import scipy.stats as stats
from traceroute import traceroute

<h1>Fijamos destino</h1>

In [ ]:
destino = 'www.google.com'

filename = 'traces/' + destino + '.csv'

IP_pub_destino = socket.gethostbyname(destino)
IP_pub_fuente = requests.get('https://api.ipify.org').text # usamos esto para conseguir nuestra ip publica para el mapa, ya que la primer direccion ip que aparece sera la ip privada del gateway

print("Vamos desde: " + IP_pub_fuente + " hasta: " + IP_pub_destino + " guardando en: " + filename)

<h1>Realizamos traceroute</h1>

In [ ]:
df = traceroute(destino, 30, 25)

<h1>Alcanzamos el destino?</h1>

In [ ]:
reached = IP_pub_destino == df['ip'].iloc[-1]
reached

<h1>Progresion de RTT total</h1>

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
df.plot(x="ttl", y="rttTot", kind="line" , title="RTT total vs TTL", ax=ax)
plt.xlabel('TTL')
plt.ylabel('RTT total (ms)')
plt.xticks(df['ttl'].tolist())
plt.legend(['RTT total'])
plt.grid(True)
# plt.savefig(filename.replace('.csv', '.svg').replace('traces/', 'svg/'))
plt.show()

<h1>Marcamos outliers (saltos interoceanicos)</h1>

<h2>Cimbala</h2>

In [ ]:
def conseguir_tau(n):
    t = stats.t.ppf(1-0.025, n-2)
    tau = t * (n - 1) / (np.sqrt(n) * np.sqrt(n - 2 + t**2))
    return tau

def marcar_outliers(df):
    try:
        datos = np.sort([rtt for rtt in df[[(not boolean) for boolean in df['outlier']]]['rttHop'].tolist() if rtt > 0])
    except:
        return df
    while len(datos) > 2:
        media = np.mean(datos)
        desvio_muestral = np.std(datos, ddof=1)
        desvio_absoluto = np.abs(datos - media)
        maximo = np.max(desvio_absoluto)
        rtt_max_des = datos[np.argmax(desvio_absoluto)]
        tau = conseguir_tau(len(datos))
        if maximo > tau * desvio_muestral:
            df.loc[df['rttHop'] == rtt_max_des, 'outlier'] = True
            datos = np.delete(datos, np.argmax(desvio_absoluto))
        else:
            return df
    return df

df = marcar_outliers(df)
df

<h1>Grafico opcional para comparar criterios de deteccion de outliers</h1>

In [ ]:
def variacion_desvio(df):
    datos = df['rttHop'].tolist()
    media = np.mean(datos)
    desvio_muestral = np.std(datos, ddof=1)
    desvio = datos - media
    variacion_desvio = desvio / desvio_muestral
    return variacion_desvio, datos

def graficar_variacion_desvio(filename, _df = None):
    if _df is None:
        df = pd.read_csv(filename)
    else:
        df = _df
    df = marcar_outliers(df)
    data_to_keep = [rtt > 0 for rtt in df['rttHop'].tolist()] 
    df2 = df[data_to_keep]
    desv,rtthops = variacion_desvio(df2)
    fig, ax = plt.subplots(figsize = (10,8))
    ttls = df2['ttl'].tolist()
    percentil = stats.norm.ppf(0.8)
    ax.set_axisbelow(True)
    plt.grid(True, axis='y')
    plt.bar(ttls, desv, color='blue', label='Variación del desvío')
    plt.axhline(percentil, color='r', linestyle='--', label='Valor de corte fijo')
    plt.axhline(0, color='black')
    for i in range(len(rtthops)):
        if df2['outlier'].iloc[i]:
            plt.bar(ttls[i], desv[i], color='red')
        if desv[i] > percentil or df2['outlier'].iloc[i]:
            plt.text(ttls[i], desv[i], "%.2f ms" %rtthops[i], ha='center', va='bottom')
    plt.xlabel('TTLs')
    plt.xticks(df2['ttl'].tolist())
    plt.ylabel('Variación del desvío')
    plt.title('Variación del desvío de TTLs, comparación con el valor de corte fijo y outliers de Cimbala')
    plt.legend(labels=['Valor de corte fijo', 'Outliers Cimbala'], 
               handles=[plt.Line2D([], [], color='r', linestyle='--'), 
                        plt.Rectangle((0, 0), 1, 1, color='red')])
    # plt.savefig(filename.replace('.csv', '_desvio.svg').replace('traces/', 'svg/'))
    plt.show()

graficar_variacion_desvio(filename, df)

<h1>Exportar dataframe de archivo</h1>

In [ ]:
# df.to_csv(filename, index=False)

<h1>Importar dataframe de archivo</h1>

In [ ]:
# df = pd.read_csv(filename)

<h1>Mapas</h1>

<h2>Estos mapas generados son muy poco confiables, por ende decidimos no incluirlos en el informe, pero se pueden utilizar en caso de querer verlos</h2>

In [ ]:
import requests
import plotly.graph_objects as go
import geopandas as gpd
from ip2geotools.databases.noncommercial import DbIpCity

<h2>Conseguimos coordenadas para mapa de ruta</h2>

In [ ]:
def get_coords(ip):
    # usamos try porque a veces la pagina no responde y algunas bloquean si se mandan muchos requests
    if ip.startswith('192.168.') or ip.startswith('10.'):
        # si es una ip privada, no se puede conseguir coordenadas, ponemos buenos aires
        return -34.613150, -58.377230
    try:
        texto = requests.get('https://www.geodatatool.com/en/?ip=' + ip, timeout=62).text.split('\n')
        lista = []
        for linea in texto:
            lista += linea.split(' ')

        for idx, palabra in enumerate(lista):
            if 'Latitude:' in palabra:
                latitud = float(lista[idx+1].replace('<span>', '').replace('</span>', '').strip())
            if 'Longitude:' in palabra:
                longitud = float(lista[idx+1].replace('<span>', '').replace('</span>', '').strip())
        if latitud == 0.0 and longitud == 0.0:
            raise Exception
        return latitud, longitud
    except:
        print('fallo geodatatool para ip: ' + ip)
        try:
            texto = requests.get('https://dazzlepod.com/ip/' + ip + '.json', timeout=62).json()
            return texto['latitude'], texto['longitude']
        except:
            try:
                response = DbIpCity.get(ip, api_key='free')
                return response.latitude, response.longitude
            except:
                return None, None

coords = {ip : get_coords(ip) for ip in tqdm(df['ip'])}
df['lat'] = df['ip'].apply(lambda x: coords[x][0])
df['lon'] = df['ip'].apply(lambda x: coords[x][1])

df

<h2>Mapa de ruta simple</h2>

In [ ]:
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

fig, ax = plt.subplots(figsize = (20,15))
countries.plot(color="lightgrey", ax = ax)

df.plot(x="lon", y="lat", kind="line" , title="test", ax=ax)
df[1:-1].plot(x="lon", y="lat", kind="scatter", title="test", ax=ax)

# make source and destination different colors and size on the scatte plot
df.loc[0:1].plot(x="lon", y="lat", kind="scatter", color="red", s=5, ax=ax)
# ojo q si reached es falso esto no es el destino sino el ultimo q respondio:
df.loc[len(df)-1:].plot(x="lon", y="lat", kind="scatter", color="lightgreen", s=5, ax=ax)

<h1>Mapa de ruta interactivo</h1>

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'ISO-3',
    lon = df['lon'],
    lat = df['lat'],
    hoverinfo = 'text',
    text = df['ttl'],
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 8,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

for i in range(len(df) - 1):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'ISO-3',
            lon = [df['lon'][i], df['lon'][i + 1]],
            lat = [df['lat'][i], df['lat'][i + 1]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
        )
    )

fig.update_layout(
    width = 1600,
    height = 1000,
    title = 'Traceroute',
    showlegend = False,
    geo = dict(
        scope = 'world',
        showland = True,
    )
)

fig.show()